# Binary
## 1-NN

In [1]:
from helper_rasar import *

df_nn1 = nn1_rasar()
df_nn1

Computing distance matrix... Wed Sep 16 12:21:43 2020
Computing 1-NN... Wed Sep 16 12:37:23 2020


idx_test  idx_neigh_train  label_train
0        19755             6992            1
1         2240            10850            1
2        12304             5194            1
3         3096             2977            0
4         7840            11798            0
...        ...              ...          ...
8855     17372            12400            1
8856      5833             2682            0
8857      4770             3559            1
8858     15334             9269            0
8859     20155            17513            1

[8860 rows x 3 columns]

## 5-NN
I choose to follow RASAR article using a 5-NN instead of 3-NN

In [1]:
from helper_rasar import *

print('Load data...', ctime())
X, y = load_data_rasar('data/lc_db_processed.csv', encoding = 'binary', seed = 42)

nn5_rasar(X, y)

Load data... Mon Sep 28 15:39:12 2020
Computing distance matrix... Mon Sep 28 15:39:13 2020
Start CV... Mon Sep 28 15:52:01 2020
...END Simple RASAR Mon Sep 28 16:00:54 2020
Accuracy: 	 0.8955210153010462, se: 0.0024668169961213535
RMSE: 		 0.32314012207503595, se: 0.0038550682317107475
Sensitivity: 	 0.9233360211596127, se: 0.001949169246478078
Precision: 	 0.9097846042570508, se: 0.0036678705596774807
Specificity: 	 0.8500172551588874, se: 0.00490448369612365


## Simple RASAR

### DF test

For each test experiments find the 0s and 1s nearest neighbors in train experiments and their relative distances.

### DF Train

For each train experiment find the 0s and 1s nearest neighbors in train experiments and their relative distance.

Since each train experiment has one label, 0 or 1, I don't want to find itself as neighbor for its respective class. So, I use a 2-NN such that if it finds out that the first NN is itself, automatically it swaps with the second. On the contrary, if it finds out that the first NN is not itself, it keeps going on with the first NN.

In [1]:
from helper_rasar import *

print('Load data...', ctime())
X_try, y = load_data_rasar('data/lc_db_processed.csv', encoding = 'binary', seed = 42)

acc, se_acc, rmse, se_rmse = cv_simple_rasar(X_try, y)

Load data... Fri Sep 18 18:23:02 2020
Computing distance matrix... Fri Sep 18 18:23:03 2020
Start CV... Fri Sep 18 18:35:24 2020
...END Simple RASAR Fri Sep 18 19:02:20 2020
Accuracy: 	 0.906997570570498, se: 0.001957554194969314
    RMSE: 		 0.30489571972672724, se: 0.003202714544842038
    Sensitivity: 	 0.9524006595432942, se: 0.0019256120362014873
    Precision: 	 0.903197085506742, se: 0.003006025789248351
    Specificity: 	 0.8325926949194414, se: 0.005033025139211729


# Multiclass

In [2]:
from helper_rasar_datafusion_multiclass import *

X, _ = load_datafusion_datasets('data/lc_db_processed.csv','data/datafusion_db_processed.csv',
                                encoding = 'multiclass', seed = 42)
y = X.conc1_mean.copy().values
del _

cv_simple_rasar_multiclass(X, y)

Computing distance matrix... Fri Oct  9 20:25:48 2020
Start CV... Fri Oct  9 20:38:52 2020
...END Simple RASAR -- Multiclass Fri Oct  9 21:08:51 2020
Accuracy: 	 0.7398121154166983, se: 0.0018281488575418079
RMSE: 		 0.6836533148079024, se: 0.007268873158371714
W. Recall: 	 0.7398121154166983, se: 0.0018281488575418079
W. Precision: 	 0.7393438049716283, se: 0.0018822492514403502


basic logistic regression

In [1]:
# sistemare h2o...
from helper_rasar_datafusion_multiclass import *

X, _ = load_datafusion_datasets('data/lc_db_processed.csv','data/datafusion_db_processed.csv',
                                encoding = 'multiclass', seed = 42)
y = X.conc1_mean.copy().values
del _

cv_simple_rasar_multiclass(X, y, final_model = False)

Computing distance matrix... Fri Oct 16 16:05:26 2020
Start CV... Fri Oct 16 16:19:27 2020
...END Simple RASAR -- Multiclass Fri Oct 16 16:49:44 2020
Accuracy: 	 0.648513457315654, se: 0.003847680219980783
RMSE: 		 0.7749638488347277, se: 0.008156360717866176
W. Recall: 	 0.648513457315654, se: 0.003847680219980783
W. Precision: 	 0.6698417659282568, se: 0.0027051642819279206


### tuning logistic regression

In [1]:
from helper_rasar_datafusion_multiclass import *
from sklearn.model_selection import train_test_split

db_mortality, db_datafusion = load_datafusion_datasets('data/lc_db_processed.csv', 'data/datafusion_db_processed.csv',
                                                      encoding = 'multiclass', seed = 42)

X = db_mortality.drop(columns = 'conc1_mean').copy()
y = db_mortality.conc1_mean.values

X_train, _, y_train, _ = train_test_split(X, y, test_size = 0.33, random_state = 42, stratify = y)

del db_mortality, _, X, y

hyper_params_tune = {'fit_intercept': [False, True],
                     'l1_ratio': [i/10 for i in range(0,11)],
                     'C': [10**i for i in range(-3,3)]}

tab_tuning_simple = cv_random_simple_rasar_multiclass(X_train, y_train, hyper_params_tune)

Computing distance matrix... Fri Oct 16 18:03:38 2020
Start CV... Fri Oct 16 18:09:00 2020
Train and test... Fri Oct 16 18:09:02 2020done... models... Fri Oct 16 18:11:18 2020done
Train and test... Fri Oct 16 18:15:27 2020done... models... Fri Oct 16 18:17:43 2020done
Train and test... Fri Oct 16 18:22:14 2020done... models... Fri Oct 16 18:24:30 2020done
Train and test... Fri Oct 16 18:29:29 2020done... models... Fri Oct 16 18:31:46 2020done
Train and test... Fri Oct 16 18:36:09 2020done... models... Fri Oct 16 18:38:27 2020done
...END Simple RASAR -- Multiclass Fri Oct 16 18:42:58 2020


In [4]:
tab_tuning_simple[tab_tuning_simple.test_acc == tab_tuning_simple.test_acc.max()] 

train_acc  test_acc  train_rmse  test_rmse  l1_ratio  fit_intercept  \
mod50   0.650450  0.647948    0.800990   0.802692       0.6          False   
mod51   0.650423  0.647948    0.801033   0.802692       0.7          False   
mod52   0.650409  0.647948    0.801176   0.802692       0.8          False   

         C  
mod50  0.1  
mod51  0.1  
mod52  0.1

In [6]:
tab_tuning_simple.to_csv('output/simple_tuning_params_multiclass.csv')

best model from CV

In [1]:
from helper_rasar_datafusion_multiclass import *

X, _ = load_datafusion_datasets('data/lc_db_processed.csv','data/datafusion_db_processed.csv',
                                encoding = 'multiclass', seed = 42)
y = X.conc1_mean.copy().values
del _

cv_simple_rasar_multiclass(X, y, final_model = True)

Computing distance matrix... Fri Oct 16 18:53:52 2020
Start CV... Fri Oct 16 19:06:00 2020
...END Simple RASAR -- Multiclass Fri Oct 16 19:35:57 2020
Accuracy: 	 0.6888915780744206, se: 0.006186046169479121
RMSE: 		 0.7340410256099156, se: 0.013300828408676144
W. Recall: 	 0.6888915780744206, se: 0.006186046169479121
W. Precision: 	 0.690326435866571, se: 0.005786091465671904


In [1]:
# abbassato la soglia di tol della regression logistica
from helper_rasar_datafusion_multiclass import *

X, _ = load_datafusion_datasets('data/lc_db_processed.csv','data/datafusion_db_processed.csv',
                                encoding = 'multiclass', seed = 42)
y = X.conc1_mean.copy().values
del _

cv_simple_rasar_multiclass(X, y, final_model = True)

Computing distance matrix... Sat Oct 17 13:00:38 2020
Start CV... Sat Oct 17 13:13:58 2020
...END Simple RASAR -- Multiclass Sat Oct 17 13:54:01 2020
Accuracy: 	 0.6897117079808112, se: 0.004954544897662281
RMSE: 		 0.7360807664134106, se: 0.011193301531053811
W. Recall: 	 0.6897117079808112, se: 0.004954544897662281
W. Precision: 	 0.691676909075827, se: 0.004490744970359585
